# Multi-Level Models (Hierarchical Models)

So far, we haven't considered the probability distribution of the fitting parameters $D_0,k_1,k_2$, and $k_3$. But to use a hierarchical model to combine our inference from different patch data and allow us to model our serum estradiol concentration, we will need to introduce hyperparameters that connect different studies. This way, our fitting parameters are given a "probabilistic specification" in terms of hyperparameters that are common for all. In our case, the hyperparameters would be the parameters of the distribution ([hyperprior](https://en.wikipedia.org/wiki/Hyperprior)) we choose to use in order to model our fitting parameters.

In [ ]:
#https://www.pymc.io/projects/examples/en/latest/generalized_linear_models/GLM-hierarchical-binomial-model.html
#https://github.com/markdregan/Bayesian-Modelling-in-Python/blob/master/Section%203.%20Hierarchical%20modelling.ipynb
#https://storopoli.io/Bayesian-Julia/

# Before and After Patch Removal

We can see from [3_Steady-State_Multi-Dose_3-Compartment](3_Steady-State_Multi-Dose_3-Compartment) that the decay after patch removal is not captured when the serum Estradiol solution didn't consider measurement after patch removal as a separate process.

After patch removal, the estradiol concentration within the depot $d_0$ (and thus $D(t_0)$) is zero.

We then [evolve](https://discord.com/channels/438306949285806082/1201973466845151362/1206123175331627029) our solution $E_2(t)$ with the initial condition at patch removal $t_{\rm rem}$ to be given by $A_{\rm eff}(t_{\rm rem})$ and $E_2(t_{\rm rem})$. i.e., we have for $E_2(t)$ after $t_{\rm rem}$

$$E_2^{t_{\rm rem}}(t)=\frac{k_2A_{\rm eff}(t_{\rm rem})}{k_2-k_3}\left[-\frac{e^{-k_2(t-T\operatorname{floor}(t/T))}}{1-e^{-k_2T}}+\frac{e^{-k_3(t-T\operatorname{floor}(t/T))}}{1-e^{-k_3T}}\right]+E_2(t_{\rm rem})\frac{e^{-k_3(t-T\operatorname{floor}(t/T))}}{1-e^{-k_3T}}.$$

In [76]:
# fitting using single-dose model
# improved, now take patch removal into account, assume removal time t_{\rm rem} = 3.5 days

def E2Dose(t, d0, k1, k2, k3):
    summation = 0
    innerFunc1 = np.exp(-k1 * t) / ((k1 - k2) * (k1 - k3))
    innerFunc2 = np.exp(-k2 * t) / ((k1 - k2) * (k2 - k3))
    innerFunc3 = np.exp(-k3 * t) / ((k1 - k3) * (k2 - k3))
    summation = d0 * (innerFunc1 - innerFunc2 + innerFunc3)
    return k1 * k2 * summation

# A_{\rm eff}(t_0) term

def AEff(t, Aeff0, k2, k3):
    summationEff = 0
    innerFunc1Eff = np.exp(-k2 * t)
    innerFunc2Eff = np.exp(-k3 * t)
    summationEff = Aeff0 *(-innerFunc1Eff + innerFunc2Eff)
    return k2 * summationEff / (k2 - k3)

# E2(t_0) term

def E2(t, E20, k3):
    return E20 * np.exp(-k3 * t)

def E2Full(t, d0, Aeff0, E20, k1, k2, k3):
    if t <= 3.5*24:
        return E2Dose(t, d0, k1, k2, k3) + AEff(t, Aeff0, k2, k3) + E2(t, E20, k3)
    else:
        return AEff(t, Aeff0, k2, k3) + E2(t, E20, k3)

In [ ]:
def emceeFit(datain, D0in, k1in, k2in, k3in):
    E2Modal = Model(E2Imp)
    Params = E2Model.make_params(d0=D0in, k1=k1in, k2=k2in, k3=k3in)
    emcee_kws = dict(steps=10000, burn=500, thin=20, is_weighted=False)
    result_emcee = E2Model.fit(data=datain[:,1], t=datain[:,0],
                                 params=Params, method='emcee',
                                 nan_policy='omit', fit_kws=emcee_kws)
    return result_emcee

In [ ]:
emceeFit()